# Growth functions {#sec-growth}

## Overview {#sec-growth-obj .unnumbered}

We review various growth functions and their applications for modeling plant growth.

By the end of this chapter, you will be able to 1) formulate dynamic plant growth models, 2) implement various growth models into computer models in Julia using *Cropbox*, 3) visualize model outcomes, and 4) evaluate model behavior.

## Readings {.unnumbered}

- (**required**) @Thornley2000 Chapter 3: Plant growth functions
- (recommended) @Yin2003: A flexible sigmoid function of determinate growth
- (recommended) @Paine2012: A comparative overview of plant growth analysis

## Questions {.unnumbered}

1.  The relative growth rate (RGR) in the form of @eq-exp-DE and @eq-semilog-exp shown below is often used in plant growth analysis. What are the pros and cons of using this approach for analyzing plant growth over time? @Paine2012 delves further into this issue as it relates to ecological research.

2.  How many parameters are there in the logistic equation @eq-logistic shown below? What are the biological meaning of each parameter when used for modeling single-plant growth and what determines their values? What will be their units?


## Simple growth models

A plant grows as it accumulates biomass ($W$) over time driven primarily by photosynthesis. This process can be expressed generically as
$W=f(t)$. Many different growth functions are available to describe the
growth patterns of plants, animals, and other organisms as an individual
or in populations. While widely used and considered useful, most of
these growth functions are empirical models unless they are founded in
underlying mechanisms and sub-processes (e.g., cell, tissue, or organ
growth). [@Thornley2000] proposes two criteria for a growth function to
be used in plant and crop modeling: 1) it should be derived from a
differential equation for $dW/dt$, and 2) the parameters should be
biologically meaningful.

### Exponential growth

The most basic form is the simple exponential growth model which has
been widely used in plant physiology and ecology for classical plant
growth analysis [@Hunt2002; @Paine2012]:

$$ 
\frac{dW}{dt} = rW
$$ {#eq-exp-DE}

where $r$ is the relative (or specific) growth rate (RGR) that
represents the proportion of new growth over existing biomass ($W$) as
dry mass (gram). Rearranging and integrating Eqn
@eq-exp-DE gives @eq-exp-growth:

$$
\begin{gathered}
    \int_{W_0}^{W}{\frac{1}{W}}\cdot dW = \int_{0}^{t}{r}\cdot  dt \Rightarrow \ln{W} - \ln{W_0} = \ln{\frac{W}{W_0}} = r\cdot t \nonumber \\
    \therefore  W = W_0 \cdot e^{rt}
\end{gathered}
$$ {#eq-exp-growth}

$W_0$ represents the initial biomass (e.g., seed weight). $W$ is the
biomass at time $t$. In the classical growth analysis the RGR ($r$) is
estimated by the linear slope of the semi-logarithmic relationship based
on biomass measurements made at two different time points (e.g., initial
and final biomass) shown in
@eq-semilog-exp.

$$
\begin{split}
    &\ln{W_2} = \ln{W_1} + r\Delta t \\
    &\therefore r = \frac{\ln{W_2}-\ln{W_1}}{t_2-t_1}
\end{split}
$$ {#eq-semilog-exp}

While useful in practice, the assumption that a plant or its population
will grow exponentially throughout their growing period is problematic.
An exponential growth is only true during early stage, and is often
followed by a linear then a saturation phase to cease the growth. This
sigmoidal growth pattern is more realistic and physiologically sound.

In @sec-intro, we've estimated the absolute growth rate (AGR; $\mathrm{m \cdot yr^{-1}}$) of Douglas-fir saplings grown in Marin County, CA in the first few decades of their life based on a figure from @Porzig2014. 

::: {#tbl-DF-height}
| Year | Height (m)
| :- | :- |
| 1985 | 1.68 |
| 1995 | 2.56 |
| 2005 | 6.99 |

  : Douglas-Fir height growth over two decades in Marin County, CA. Data
  from [@Porzig2014]
:::

::: {#exr-DF-exp}
### Douglas-fir exponental growth phase

In the previous chapter, we've estimated the absolute growth rate (AGR; m ^−1^) of Douglas-Fir saplings grown in Marin County, CA in the
first few decades of their life based on a figure from [@Porzig2014](@exr-DF-early-growth).

@tbl-DF-height provides mean tree heights over
20 year period with 10 year intervals. Calculate the relative growth
rate (RGR; $r$) for each 10 year period (i.e., 1985 - 1995 and 1995 -
2005) and over the entire 20 year period (1985 - 2005), and compare
their values. These trees are in relatively early stage of their growth.
Do you think the exponential growth model does a good job of describing
their growth pattern? How do RGRs from this problem compare with the AGR
from @exr-DF-early-growth in @sec-intro? What are their units?
:::

:::{#nte-symbol-H .callout-note appearance="simple"}
For Douglas-fir height growth example, we use symbol $H$ to represent *height* instead of $W$ which is commonly used to represent *biomass*.
:::
Exact age of the trees when the measurements were made was unknown. But the mean initial height ($H_0$) of the trees was 1.42m in 1983 when the first measurements were made. We will create a function called `r` to calculate RGR using the semi-log function for Douglas-fir height data. Note that we are ignoring units here but an important feature of `Cropbox` is the accounting of all units used in the model. We will revisit this later.

::: {#lst-DF-RGR}

In [ ]:
#| output: false
r(t1, t2; H1, H2) = (log(H2)-log(H1))/(t2-t1)

In [ ]:
#| output: false
r(1985,2005; H1 = 1.68, H2 = 6.99)

In [ ]:
using Cropbox, CSV, DataFrames

In [ ]:
#| output: false
@system DF_exp(Controller) begin
    H0      => 1.42  ~ preserve
    r       => 0.071  ~ preserve(parameter)
    H(r,H)  => r*H   ~ accumulate(init = H0)
    yr      => 1     ~ accumulate(init = 1983)
end

In [ ]:
#| output: false
visualize(DF_exp, :yr, :H; stop = 27, kind = :line)

In [ ]:
#| output: false
DF_obs = CSV.read("./data/PSME-height.csv", DataFrame) |> unitfy; 
# keeping the correct unit is a key in Cropbox. We will talk more about this.

In [ ]:
#| output: false
p = visualize(DF_obs, :year, :height; )
visualize!(p, DF_exp, :yr, :H; stop=27, kind = :line)

Douglas-fir early height growth
:::

Here we've overlaid the digitized data with our model. How did did the model do? How does it compare with the linear model we worked in the previous chapter? Which model would you choose to use and why?

### Logistic growth

One of the most widely used growth models is the logistsic function for its intuitive parameters that are relatable to biological traits and ecological concepts. For example, if we assume that a plant grows through cell division and expansion at a rate proportional to the existing cells and their biomass with little to no physical or physiological limitations, the early growth rate is likely to resemble that of the exponential growth growth. However, as the plant grows bigger and taller the growth rate will slow down. This may be because the plant is approaching its ceiling for height and biomass that's controlled by genetic, physiological, or physcal limits (e.g., potential biomass, $W_f$; maximal height, $H_f$). The logistic function provides a useful mathematical formula to describe this growth pattern. The plant height growth may be written as


$$
\frac{dH}{dt} = rW(1-\frac{W}{W_f}) = rW(\frac{W_f-W}{W_f})
$$ {#eq-logi-DE}

$r$ here represents the relative growth rate, as in @eq-exp-DE, but it is strictly applicable to the exponential phase only that is not realized in any other growth phase. Biomass is represented by $W$, final (or potential) biomass by $W_f$, and initial biomass by $W_0$. If the final (or potential) biomass ($W_f$) is known, the rate of growth would slow down as the biomass ($W$) approaches $W_f$.

@eq-logi-DE can be integrated to give:

$$
\begin{split}
    W &= \frac{W_0 W_f}{W_0 + (W_f-W_0)~e^{-rt}} = \frac{W_f}{1 + (\frac{W_f}{W_0}-1)~e^{-rt}}
\end{split}
$$ {#eq-logistic}


There are a few variants of the integral form of the logistic equation.
Two of them are shown in
@eq-logistic. The logistic function is more likely
to represent how a plant grows and reaches its final size than the
exponential function, and works particularly well for many annual
plants.

![Douglas-fir height (a) and height growth rate (b) over 200 years.
Source: [@Bond2007]](./figs/Bond-2007-fig1-DF_height.png){#fig-Bond-DF-growth
width="60%"}

::: {#exr-DF-logistic}
@Bond2007 illustrated that a Douglas-Fir tree can
grow up to 70m in height when mature (@fig-Bond-DF-growth) but their final height will depend on the site quality. With this information, let's do the following:

$$
\frac{dH}{dt} = rH(1-\frac{H}{H_f}) = rH(\frac{H_f-H}{H_f})
$${#eq-logi-DE-H}

Where $H$ represents tree height, $H_f$ is final (or potential) height, and $H_0$ is initial height.

1.  Create a `Cropbox` system that simulates Douglas-Fir tree growth at a high quality site over 200 years using the logistic differential equation (@eq-logi-DE-H)

2.  Recreate Figure 2.1a for high quality site

3.  Recreate Figure 2.1b for high quality site

4.  Compare your results with the original figures and discuss. How would you grade this model's performance to describe Douglas-fir growth and why?
:::

:::{#lst-df-logistic}

In [ ]:
#| output: false
@system DF_logi(Controller) begin
    H0         => 1.42               ~ preserve
    r          => 0.07               ~ preserve(parameter)
    Hf         => 70.0               ~ preserve(parameter)
    H(r,H,Hf)  => r*H*(1.0 - H/Hf)   ~ accumulate(init = H0)
    yr         => 1                  ~ accumulate(init = 1983)
    dH(r,H,Hf) => r*H*(1.0 - H/Hf)   ~ capture 
    # This "state verb" or command captures the changes occuring at each time step
end

In [ ]:
#| output: false
p = visualize([DF_logi], :time, :H; stop=200, kind = :line)

In [ ]:
#| output: false
p = visualize([DF_logi], :H, :dH; stop=200, kind = :line)

Doulas-fir logistic height growth
:::

### Gompertz function

The growth of a plant will slow down if its efficiency of synthesising new tissue declines (or inefficiency increases) even if the resources are non-limiting. For example, a very tall tree may slow down its height growth because of the physical constraints (i.e. reduced transport efficiency) it is facing in getting the resources (e.g., water) to where they are needed - the meristems, even if it is physiologically young and active and the resources like water and nitrogen are abundant in the soil. This transport inefficiency increases over time as the tree gets taller. As a result of this constraint, the growth efficiency (RGR) declines correspondingly over time. This case can be expressed as:

$$
\begin{split}
    &\frac{dW}{dt} = rW \\
    &\frac{dr}{dt} = -{\alpha}r
\end{split}
$${#eq-gompertz-de}

where $W$ represents biomass, $r$ is the relative growth rate or
specific growth rate, and $\alpha$ is the decay rate of growth
efficiency over time. This set of differential equations can be
integrated into what is known as the Gompertz equation. The Gompertz
growth model is a popular choice for modeling tumor growth in oncology
by assuming that the growth rate declines as the cell mass multiplies
[@MathBiol].

$$
W=W_0\exp{\left[\frac{r_0}{\alpha}(1-e^{-\alpha{t}})\right]}
$${#eq-gompertz}

The flexibility of the Gompertz function is remarkable [@Thornley2007]
because the final output can vary depending on how the growth conditions
influence $r$ and $\alpha$ in the differential form of the equation.
However, a problem is that the growth rate begins to decline immediately
at $t$ = 0 while in most organisms the onset of this decline or ageing
may be delayed until later in the life stages. The flexible open form of
the equation @eq-gompertz-de could allow for remedying this situation.

::: {#exr-Gompertz}
### The Gompertz model
Using `Cropbox` framework, create systems that implement 1) differential and 2) integral forms of the Gompertz function. 

1. Simulate both forms of the model for 300 days using daily steps starting with the parameter values provided in @tbl-logi-parms. Adjust parameters as needed to approximat the model behavior to mimic the logistic growth. Plot and compare their results. 

2. Run the differential form of the models to run on daily time step, every 12 hours, every hour, and every minute, compare the results, and discuss why there are differences.
:::

## Parameters and variables

Now let's think about the parameters and variables of the logistic growth model (@eq-logistic). What is the state variable? How many parameters are there? Do the parameters have biological meaning? Are they measurable, observable, or at least imaginable? $W$ is the plant biomass (dry mass in gram) whose state changes over time as plant grows. Thus, it is a *state variable* of this model. $W_0$ is the initial biomass that can be determined by measuring seed mass (g). It has clear meaning and unit, and can be assumed to remain the same for a species or cultivar; it is a *parameter* in this model system. How about $r$ defined previously as 'relative growth rate'? It is likely to remain unchanged for the same plant or genotype (e.g., clone, cultivar, species) can be characterized by its genetics. It will be influenced by its environment but if we assume a constant environment, then it is a unique value for the plant that doesn't change over time. So, it's another *parameter* here. How can we determine the value of $r$? This is where calibration or model fitting comes in to play a role. Note that the actual rate of growth as *rate* represented by $dW/dt$ is not identical to $r$ because it slows down over time as the plant reaches its final size or biomass ($W_f$). The final biomass $W_f$ is an interesting *parameter* that may or may not be easily determined. If it is used to represent the potential biomass of a plant that can be reached only in an optimal growth condition with unlimited resources, then it should signify the genetic ceiling of the plant. How do we measure that in practice or can we ever? In practice, we can determine the values of these parameters ($W_0, r, W_f$) through *model calibration* process using some sort of optimization techniques especially if we have sufficient observational data over a wide range of plants' growth stages. Therefore, it is often useful to have observations spread over time than having more observations concentrated only on few time points. For example, if you have limited time and resources to make only 100 observations, then having 5 reps over 20 time points will work better for calibrating (i.e., fitting) the model to estimate parameter values than having 20 reps over 5 times or 50 reps over only 2 times. This is even more true if a model is complex with many parameters to fit. Whether or not the observations we have best represent the true meaning of the model parameters is something to think about. See@Paine2012 for further discussion.

See @tbl-logi-parms for summary of model parameters mentioned in this discussion. For additional features and analysis of this equation and other growth functions, see Ch 3 of @Thornley2000. The Gompertz, Chanter, and Richards equations are other growth functions depicting the sigmoidal growth patterns with biologically meaningful parameters, representing different assumptions and constraints of growth. See Ch 3 of @Thornley2000 for details. Another interesting growth function is one derived from the beta distribution model @Yin2003.


::: {#tbl-logi-parms}
| Symbol | Value | Units | Description |
| :- | :- | :- | :- |
| $t$ | - | $\mathrm{d}$ | Time unit used in the model |
| $r$ | 0.05 | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Relative (or specific or intrinsic) growth rate |
| $W_0$ | 0.25 | $\mathrm{g}$ | Seed mass or initial biomass |
| $W_f$ | 300 | $\mathrm{g}$ | Potential final mass of a plant |
| $W$ | - | $\mathrm{g}$ | Whole-plant biomass as dry matter (state variable) |

Logistic model variables and parameters for an annual crop biomass accumulation
:::

::: {#exr-corn-logistic}
### Logistic growth of an annual crop

Implement the logistic growth function in to two different systems using *Cropbox* to describe the growth of a productive annual crop such as corn or sorghum: 1) logistic differential equation @eq-logi-DE and 2) analytical solution @eq-logistic. Applying the parameter and initial values in @tbl-logi-parms, simulate both forms of the logistic model for 300 days on *daily time step*. Plot to overlay the two forms of the model in one figure and compare the results. Before running simulations, predict if you expect the results from both models to be identical, and discuss why or why not. Did your simulations support your predictions? Why or why not? Test the behavior of the model with respect to its parameter values.
:::

Here is the model specification of @eq-logi-DE implemented on Cropbox framework for @exr-corn-logistic. See how variables and parameters are declared each line with state verbs (`track`, `preserve`, `accumulate`) and tags (*i.e.* `parameter`, `init`, `u".."`).

- Time variable `t` is set to `track` internal time variable (`context.clock.tick`) in the unit of `d` (day) which is a conversion from default `hr` (hour).
- Parameters `r`, `W0`, `Wf` are set to `preserve` their initial values which are declared in the body. They are tagged `parameter` indicating their values may be replaced by configuration when running simulations.
- As the state variable `W` is defined as a form of derivative ($\frac{dW}{dt}$), we want to integrate its value over time. As Cropbox is essentially a discrete-time modeling framework, what we do here is `accumulate` (summing up) instead of integration over continous time.

In [ ]:
#| output: false
@system crop_diff(Controller) begin
    t(context.clock.tick) ~ track(u"d")
    
    r: relative_growth_rate => 0.05 ~ preserve(u"g/g/d", parameter)
    W0: initial_biomass => 0.25 ~ preserve(u"g", parameter)
    Wf: potential_final_biomass => 300 ~ preserve(u"g", parameter)
    
    W(r, W, Wf): biomass => begin
        r * W * (Wf - W) / Wf
    end ~ accumulate(u"g", init=W0)
end

`simulate(<System>; ..)` function make an instance of the model system and run simulation until `stop` condition is met. Here we use `300u"d"` as a stop condition which indicates the simulation should run for 300 days. The output of `simulate()` is a table-like DataFrame with columns from variables specified in the model.

In [ ]:
#| output: false
r1 = simulate(crop_diff; stop=300u"d")

We can draw a graph with `plot(<DataFrame>, <x>, <y>; ..)` function with the DataFrame output. By default, `kind` option is set to `:scatter` which could be inefficient to plot too many data points, so we instead decided to use `:line` to draw a line plot.

In [ ]:
#| output: false
plot(r1, :t, :W; kind=:line)

::: {#tip-clock .callout-tip}
### **Cropbox**: `Clock` and `config`

Note that models in `Cropbox` have a default clock set in hourly time step (`context.clock.step`). We may change the time step by providing a user configuration. Configuration is in a two-level mapping like `<System> => <Variable> => <Value>`. Cropbox supports many different ways of constructing construction and one simple way is to use a pair of pair.

In [ ]:
#| output: false
c = :Clock => :step => 1u"d"

Here we make a configuration `c` for system `Clock` and want to have its variable `step` to be `1u"d"` (1 day). We can take a look at how this pair of pair structure transformed into an actual instance of configuration by calling `configure()` or `@config`.


In [ ]:
#| output: false
@config(c)

Now we re-run the simulation with the custom configuration `c` for daily time step.

In [ ]:
#| output: false
r1 = simulate(crop_diff; config=c, stop=300u"d")

In [ ]:
#| output: false
plot(r1, :t, :W; kind=:line)

:::

Specification of the system for analytical solution (@eq-logistic) looks very similar to the differential form (@eq-logi-DE) except the state variable `W` is already in the form of integral, not a derivative. Thus, what we need to do is just to make sure re-calculate `W` to `track` its value every time step instead of accumulation / integration.

In [ ]:
#| output: false
@system crop_sln(Controller) begin
    t(context.clock.tick) ~ track(u"d")
    
    r: relative_growth_rate => 0.05  ~ preserve(u"g/g/d", parameter)
    W0: initial_biomass => 0.25 ~ preserve(u"g", parameter)
    Wf: potential_final_biomass => 300 ~ preserve(u"g", parameter)
    
    W(r, W0, Wf, t): biomass => begin
        Wf / (1 + (Wf/W0 - 1)* exp(-r*t))
    end ~ track(u"g")
end

In [ ]:
#| output: false
r2 = simulate(crop_sln; config=c, stop = 300u"d")

In [ ]:
#| output: false
plot(r2, :t, :W; kind=:line)

:::{#tip-overlay .callout-tip}
### Cropbox: Plot overlay

We can overlay multiple plots with consecutive calls of `plot!(<Plot>, ...)` with a Plot object returned by previous `plot()` or `plot!()` function.

In [ ]:
#| output: false
p = plot(r1, :t, :W; kind=:line, name="Differential Equation")
plot!(p, r2, :t, :W; kind=:line, name="Analytica Solution")

Cropbox also provides some high-level plotting functions to make a plot without manually running simulations first. One form of `visualize()` function supports `visualize([<System>..], <x>, <y>; <simulate() options>.., <plot() options..>)` to make an overlay plot in one line.


In [ ]:
#| output: false
visualize([crop_diff, crop_sln], :t, :W; config=c, stop=300u"d", kind=:line)

You may notice a small difference between two implementations of logistic growth models. Answer the question above and discuss with others.
:::

## Homework Problems {#sec-growth-hw}
Submit your response to all problems in a single Jupyter Notebook
(.ipynb file) via Canvas in a week.

Based on Ch 3 of @Thornley2000, **1)** create a system
using *Cropbox* for each of the following growth models (i.e, Monomolecular, Gompertz, and Chanter functions) in the form of **1a)** differential equation and **1b)** analytical solutionusing the crop biomass growth example(@exr-corn-logistic). Run simulations on  *daily time step*, **2)** plot both differential form and analytical solution of each model in one figure by overlaying two lines as done in the lab exercises, **3)** examine the model behavior with different input conditions and parameter values and
**4)** describe any notable observations, and interpret discuss the
results.

1.  Monomolecular equation
2.  Gompertz equation
3.  Chanter equation